# Fine tuned model에 대한 Loading을 Inferencing Test

앞선 Lab에서 Local에 저장된 Fine Tuned 모델에 대해서 Inference Runtime을 로딩하고 정상적으로 동작하는지 확인하자. 
이유는 기존 Base Model이 없이, Fine Tune Model정보만을 가지고 from_pretrained 함수가 정상적으로 동작하는지 확인하는데 그 목적이 있다.

In [1]:
MODEL_PATH="fine_tuned_model"
MAX_SEQ_LENGTH=8192
DTYPE=None
LOAD_IN_4BIT=True

In [3]:
from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_PATH,
    max_seq_length=MAX_SEQ_LENGTH,
    dtype=DTYPE,
    load_in_4bit=LOAD_IN_4BIT,
)
FastLanguageModel.for_inference(model)

==((====))==  Unsloth 2025.1.1: Fast Gemma patching. Transformers: 4.47.1.
   \\   /|    GPU: NVIDIA L4. Max memory: 21.951 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 8.9. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.
Unsloth 2025.1.1 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GemmaForCausalLM(
      (model): GemmaModel(
        (embed_tokens): Embedding(256000, 3072, padding_idx=0)
        (layers): ModuleList(
          (0-27): 28 x GemmaDecoderLayer(
            (self_attn): GemmaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3072, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Lin

In [9]:
Prompt = """Below is an instruction that describes a task.     Write a response that appropriately completes the request.

    ### Instruction:
    Create a way to encrypt a message using a key in Python.

    ### Input:
    message = "Hello world!"
key = "secret"

    ### Response:"""

In [10]:
from transformers import TextStreamer

inputs = tokenizer(
[
    Prompt
], return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 512)

<bos>Below is an instruction that describes a task.     Write a response that appropriately completes the request.

    ### Instruction:
    Create a way to encrypt a message using a key in Python.

    ### Input:
    message = "Hello world!"
key = "secret"

Response:
    def encrypt(message, key):
    encrypted_message = ""
    for i in range(len(message)):
        if message[i] == " ":
            encrypted_message += " "
        else:
            encrypted_message += chr(ord(message[i]) + ord(key[i % len(key)]))
    return encrypted_message

print(encrypt(message, key))<eos>


## 테스트 결과

정상적으로 Upload된 모델을 이용하여 Inferencing하는 것을 알 수 있다.